# Importação de bibliotecas e carregamento da base de dados

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.neighbors import NearestNeighbors

# Carregar interações
df = pd.read_csv('C:\\Users\\joao.pineda\\Downloads\\goodreads_interactions.csv')

# Visualização inicial
print("\n=== Amostra dos Dados ===")
print(df.head())
print("\n=== Metadados ===")
print(df.info())
print("\n=== Estatísticas Descritivas ===")
print(df.describe())


=== Amostra dos Dados ===
   user_id  book_id  is_read  rating  is_reviewed
0        0      948        1       5            0
1        0      947        1       5            1
2        0      946        1       5            0
3        0      945        1       5            0
4        0      944        1       5            0

=== Metadados ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype
---  ------       --------------    -----
 0   user_id      1048575 non-null  int64
 1   book_id      1048575 non-null  int64
 2   is_read      1048575 non-null  int64
 3   rating       1048575 non-null  int64
 4   is_reviewed  1048575 non-null  int64
dtypes: int64(5)
memory usage: 40.0 MB
None

=== Estatísticas Descritivas ===
            user_id       book_id       is_read        rating   is_reviewed
count  1.048575e+06  1.048575e+06  1.048575e+06  1.048575e+06  1.048575e+06
mean   1.058027e+0

# Tratamento da Base de Dados

In [15]:
# Limpeza inicial
print("\n=== Etapa de Limpeza ===")
print(f"Registros iniciais: {len(df):,}")
df = df.drop_duplicates(subset=['user_id', 'book_id'])
df = df[df['rating'] > 0]
df = df.groupby('user_id').filter(lambda x: len(x) >= 3)
print(f"Registros após limpeza: {len(df):,}")

# Divisão treino-teste
splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(splitter.split(df, groups=df['user_id']))
train, test = df.iloc[train_idx], df.iloc[test_idx]
print(f"\nDivisão Treino-Teste: {len(train):,} | {len(test):,}")


=== Etapa de Limpeza ===
Registros iniciais: 1,048,575
Registros após limpeza: 493,226

Divisão Treino-Teste: 390,948 | 102,278


# Engenharia de Atributos

In [16]:
# Matriz usuário-item
user_item_matrix = train.pivot_table(
    index='user_id',
    columns='book_id',
    values='rating',
    fill_value=0
)

# Filtragem Colaborativa

In [17]:
# Treinamento do modelo
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(user_item_matrix)

# Função de recomendação
def recommender(user_id, k=10):
    try:
        user_vector = user_item_matrix.loc[user_id].values.reshape(1, -1)
        _, indices = knn.kneighbors(user_vector)
        return user_item_matrix.columns[indices.flatten()].tolist()[:k]
    except KeyError:
        return []  # Fallback para usuários não vistos

# Avaliação do Modelo

In [18]:
def evaluate_model(test_data, k=10):
    precisions = []
    recalls = []
    
    for user in test_data['user_id'].unique():
        true_books = test_data[test_data['user_id'] == user]['book_id'].tolist()
        recommended = recommender(user, k)
        
        # Cálculo de acertos
        hits = len(set(true_books) & set(recommended))
        
        # Métricas por usuário
        prec = hits / k if k > 0 else 0
        recall = hits / len(true_books) if len(true_books) > 0 else 0
        
        precisions.append(prec)
        recalls.append(recall)
    
    return np.mean(precisions), np.mean(recalls)

# Cálculo e exibição das métricas
precision, recall = evaluate_model(test)

print("\n" + "="*50)
print(f" Precision@10: {precision:.4f} ({(precision*100):.1f}%)")
print(f"    Recall@10: {recall:.4f} ({(recall*100):.1f}%)")
print("="*50)
print("\nInterpretação:")
print(f"- Em média, {precision*100:.1f}% dos livros recomendados são relevantes")
print(f"- O modelo recupera {recall*100:.1f}% dos livros que o usuário realmente gostaria de ler")



 Precision@10: 0.0000 (0.0%)
    Recall@10: 0.0000 (0.0%)

Interpretação:
- Em média, 0.0% dos livros recomendados são relevantes
- O modelo recupera 0.0% dos livros que o usuário realmente gostaria de ler
